<a href="https://colab.research.google.com/github/ZS4MLDL/learn_pytorch/blob/main/Transformer_Scratch_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, randomsplit
from torch.utils.tensorboard import SummaryWriter
#Math
import math
#Hugging Face
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevel
from tokenizers.pre_tokenizers import Whitespace

from pathlib import path
from tqdm import tqdm
from typing import Any
import warnings

!pip install datasets

In [ ]:
class InputEmbedding(nn.Module):
  def __init__(self, d_model:int, vocab_size:int):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
      return self.embedding(x) * math.sqrt(self.d_model)

In [ ]:
class P